In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'  #'0,1,2,3,4,5,6,7'
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments, ViTImageProcessor,AutoImageProcessor
import os
from PIL import Image
from sklearn.metrics import accuracy_score
import numpy as np
import pyarrow.parquet as pq
import io
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from peft import LoraConfig, TaskType, get_peft_model,PeftModel, PeftConfig
#from transformers import LoRAConfig, LoRAAdapter
from utils import get_mnist_data, get_EuroSAT_data,CustomTensorDataset, get_cifar10_data, get_car_data,get_fruits_data, get_GTSRB_data, get_DTD_data, get_resis_data, get_grabage_data, get_plants_data
import random
from tqdm import tqdm
import torch.nn.functional as F

def set_random():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    # 如果使用 GPU，还需固定 GPU 相关随机数
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # 用于多 GPU 情况
    # 确保卷积操作等确定性（针对特定卷积算法）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def test_result():
    sum1 = 0
    set_random()
    for i in tqdm(range(min(1000, len(test_dataset)))):
        with torch.no_grad():
            outputs = model(test_dataset[i]["pixel_values"].unsqueeze(0).cuda())
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()
        if predicted_class != int(test_dataset[i]['labels']):
            # print(predicted_class, int(test_dataset[i]['labels']))
            sum1 += 1
    print(1-sum1/min(1000, len(test_dataset)))
    return 1-sum1/min(1000, len(test_dataset))
    

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}
model_dir = "./vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_dir, torch_dtype="auto")
feature_extractor = ViTImageProcessor.from_pretrained(model_dir)


lora_config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=[ "intermediate.dense", "output.dense"],  #["query", "value"]
        lora_dropout=0.1,
        bias="none",
        modules_to_save=["classifier"],
    )

model = get_peft_model(model, lora_config).cuda()
# model.print_trainable_parameters()


/home/xiaoyuanxin/miniconda3/envs/lyf_jiuan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train_dataset, test_dataset = get_GTSRB_data()  
# train_dataset, test_dataset = get_DTD_data()
# train_dataset, test_dataset = get_resis_data()  
# train_dataset, test_dataset = get_grabage_data() 
train_dataset, test_dataset = get_mnist_data()   #select task you train

In [ ]:

training_args = TrainingArguments(
    output_dir=f"./{task}_checkpoint",          # 输出目录
    #evaluation_strategy="epoch",    # 每个 epoch 后评估一次
    #save_steps=10,
    save_total_limit=1,
    learning_rate=5e-3,              # 学习率2e-5
    per_device_train_batch_size=64, # 训练批次大小
    per_device_eval_batch_size=64,  # 验证批次大小
    num_train_epochs=1,             # 训练轮数
    weight_decay=0.01,              # 权重衰减
    logging_dir="./logs",           # 日志目录
    logging_steps=10,               # 每 10 步记录一次日志
)

trainer = Trainer(
    model=model,                         # 模型
    args=training_args,                  # 训练参数
    train_dataset=train_dataset,         # 训练集
    eval_dataset=test_dataset,            # 验证集
    tokenizer=feature_extractor,         # 特征提取器（用于处理输入）
    compute_metrics=compute_accuracy      # 评估指标
)

# 开始训练

trainer.train()

In [ ]:
test_result()